# Un premier graphe de compositeurs

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import networkx as nx
import spacy

## Récupération des noms depuis Classic Cat

In [7]:
url = requests.get("https://www.classiccat.net/composers.php")
page = BeautifulSoup(url.content, "html.parser")
li_compo = page.find_all("a", href = True)
compos = []
for lien in li_compo:
    text = lien.get_text()
    if "," in text:
        if len(compos)>0:
            if text[0] < compos[-1][0]:
                break
        compos.append(text)


In [8]:
composers = [(compo.split(",")[1],compo.split(",")[0])  for compo in compos]

## Complétion des données à partir de Wikidata

### Méthode obsolète

In [ ]:
def find_info(soup, prop, dico, name):
    """Actualise le dictionnaire dico avec la première valeur de la propriété prop trouvée sur wikidata"""
    bloc = soup.find("div", id = prop)
    if bloc is None:
        res = None
    else:
        group = bloc.find("a",href=lambda href: href and href.startswith('/wiki/Q'))
        res = group['title'], group.get_text()
    dico[name]=res

In [ ]:
def complete_infos_all(name,dict_props):
    """Complète toutes les informations d'un compositeur à partir de son nom et de ses propriétés.
    Attention, le prédicat doit comporter un lien hypertexte"""
    dico = {"prénom" : name[0], "nom" : name[1]}
    url = f"https://www.wikidata.org/w/index.php?search={name[0]}+{name[1]}&title=Special%3ASearch&ns0=1&ns120=1"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    bloc = soup.find("div", class_ = "mw-search-result-heading")
    url = bloc.find("a")["href"]
    id = url[6:]
    dico["id"] = id
    page = requests.get("https://www.wikidata.org"+url)
    soup = BeautifulSoup(page.content, "html.parser")
    for nom,prop in dict_props.items():
        find_info(soup, prop, dico, nom)
    return dico

In [ ]:
li_props = {'nationalité' : 'P27',"sexe":"P21"}

In [73]:
complete_infos_all(("Johann-Sebastian","Bach"),li_props)

{'prénom': 'Johann-Sebastian',
 'nom': 'Bach',
 'id': 'Q1339',
 'nationalité': ('Q696651', 'Saxe-Eisenach'),
 'sexe': ('Q6581097', 'male'),
 'date de naissance': ('Q11148', 'The Guardian'),
 'date de décès': ('Q36578', 'Integrated Authority File')}

### Une autre méthode plus efficace

In [13]:
def find_id(name):
    """Récupère l'id d'un compositeur à partir de son nom"""
    url = f"https://www.wikidata.org/w/index.php?search={name[0]}+{name[1]}&title=Special%3ASearch&ns0=1&ns120=1"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    bloc = soup.find("div", class_ = "mw-search-result-heading")
    group = bloc.find("a")
    if group is None:
        return None
    else:
        return group["href"][6:]

In [14]:
find_id(("Johann-Sebastian","Bach"))

'Q1339'

In [ ]:
def actualise_prop(dico,prop,nameprop,id):
    """Actualise le dictionnaire avec la propriété prop"""
    sparql_endpoint = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?prop ?propLabel WHERE {{
        wd:{id} wdt:{prop} ?prop.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }}
    }}
    """
    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'})
    li_rep = []
    if response.status_code == 200:
        data = response.json()
        for item in data['results']['bindings']:
            li_rep.append((item['prop']['value'][31:],item['propLabel']['value']))
        dico[nameprop] = li_rep
    else:
        dico[nameprop] = None
        print("Erreur lors de la requête SPARQL:", response.status_code)

In [19]:
name = ("Johann-Sebastian","Bach")
id = find_id(name)
dico = {}
actualise_prop(dico,'P27','pays',id)
dico

{'pays': [('Q12548', 'Saint-Empire romain germanique'),
  ('Q696651', 'duché de Saxe-Eisenach')]}

In [27]:
dico_props = {"pays" : "P27","date de naissance" : "P569", "genre" : "P136", "influences" : "P737", "oeuvres":"P800", "religion" : "P140", "étudiant de" : "P1066",
              "professeur de": "P802", "écoles" : "P69", "domaines" : "P101", "institut" : "P108", "début de carrière" : "P2031"}

In [28]:
def create_profile(name,dico_props):
    """Créé le profil d'un compositeur à partir des propriétés voulues et de son nom"""
    dico = {"prénom" : name[0], "nom" : name[1]}
    id = find_id(name)
    dico["id"] = id
    if id is None:
        print("Erreur, aucun dictonnaire renvoyé")
    for nom,prop in dico_props.items():
        actualise_prop(dico, prop, nom, id)
    return dico

In [25]:
create_profile(("Johann-Sebastian","Bach"),dico_props)

{'prénom': 'Johann-Sebastian',
 'nom': 'Bach',
 'id': 'Q1339',
 'pays': [('Q12548', 'Saint-Empire romain germanique'),
  ('Q696651', 'duché de Saxe-Eisenach')],
 'date de naissance': [('', '1685-03-31T00:00:00Z')],
 'genre': [('Q8361', 'musique baroque')],
 'influences': [('Q1340', 'Antonio Vivaldi'),
  ('Q76485', 'Johann Pachelbel'),
  ('Q508635', 'Johann Georg Pisendel')],
 'oeuvres': [('Q75464', 'Sonates et partitas pour violon seul'),
  ('Q208224', 'concertos brandebourgeois'),
  ('Q211971', 'Le Clavier bien tempéré'),
  ('Q213728', 'Variations Goldberg'),
  ('Q310549', 'Messe en si mineur'),
  ('Q379111', 'Passion selon Saint Matthieu'),
  ('Q392734', 'Toccata et fugue en ré mineur'),
  ('Q465878', "L'Art de la fugue"),
  ('Q642010', 'Oratorio de Noël'),
  ('Q693675', "L'Offrande musicale"),
  ('Q756843', 'Suites pour violoncelle seul'),
  ('Q865333', 'Passion selon saint Jean'),
  ('Q892166', 'Jesu, meine Freude'),
  ('Q908241', 'Sonates en trio pour orgue de Bach'),
  ('Q1053305